# Set up

In [ ]:
!git clone https://github.com/NLP-Reichman/assignment_1.git
!mv assignment_1/data data
!rm assignment_1/ -r

Cloning into 'assignment_1'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 96 (delta 30), reused 64 (delta 13), pack-reused 0
Receiving objects: 100% (96/96), 6.77 MiB | 13.01 MiB/s, done.
Resolving deltas: 100% (30/30), done.


# Introduction
In this assignment you will be creating tools for learning and testing language models. The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.
The relevant files are under the data folder:

- en.csv (or the equivalent JSON file)
- es.csv (or the equivalent JSON file)
- fr.csv (or the equivalent JSON file)
- in.csv (or the equivalent JSON file)
- it.csv (or the equivalent JSON file)
- nl.csv (or the equivalent JSON file)
- pt.csv (or the equivalent JSON file)
- tl.csv (or the equivalent JSON file)

In [7]:
import json
# from google.colab import files
import pandas as pd
import numpy as np

# Implementation

## Part 1
Implement the function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. Our token definition is a single UTF-8 encoded character. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

Note - do NOT lowercase the sentences in whi HW.

In [2]:
START_TOKEN = "<start>"
END_TOKEN = "<end>"
UNKNOWN_TOKEN = "<unk>"

def get_texts_for_lang(lang: str) -> list[str]:
    '''
    Return a list of texts for the given language
    :param lang: the language of the texts
    :return: a list of texts
    '''
    data_file = f"data/{lang}.csv"
    df = pd.read_csv(data_file, encoding="utf-8")
    return df["tweet_text"].tolist()


In [3]:
def preprocess() -> list[str]:
    '''
    Return a list of characters, representing the shared vocabulary of all languages
    '''
    languages = ["en", "es", "fr", "in", "it", "nl", "pt", "tl"]
    vocab = set()
    vocab.add("<start>")
    vocab.add("<end>")
    for lang in languages:
        texts = get_texts_for_lang(lang)
        for text in texts:
            for c in text:
                vocab.add(c)
    return list(vocab)

vocabulary = preprocess()

## Part 2
Implement the function *lm* that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant *n*-1 sequences, and the values are dictionaries with the *n*_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{ "ab":{"c":0.5, "b":0.25, "d":0.25}, "ca":{"a":0.2, "b":0.7, "d":0.1} }

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [4]:
def lm(lang: str, n: int, smoothed: bool = False) -> dict[str, dict[str, float]]:
    '''
    Return a language model for the given lang and n-gram (n), with an option for smoothing.
    :param lang: the language of the model
    :param n: the n_gram value
    :param smoothed: boolean indicating whether to apply smoothing
    :return: a dictionary where the keys are n-1 grams and the values are dictionaries
    '''
    '''
    Return a language model for the given lang and n_gram (n)
    :param lang: the language of the model
    :param n: the n_gram value
    :return: a dictionary where the keys are n_grams and the values are dictionaries
    '''
    texts = get_texts_for_lang(lang)
    
    model = dict()

    for text in texts:
        for i in range(0,n-1,1):
            token = START_TOKEN * (n-i-1) + text[0:i]
            if token not in model.keys():
                model[token] = {}
            if text[i] not in model[token].keys():
                model[token][text[i]] = 1
            else:
                model[token][text[i]] += 1 
        for i in range(len(text)-n+1):
            token = text[i:i+n-1]
            if token not in model.keys():
                model[token] = {}
            if text[i+n-1] not in model[token].keys():
                model[token][text[i+n-1]] = 1
            else:
                model[token][text[i+n-1]] += 1 
        # end of sentence
        token = text[i+1:i+n]
        if token not in model.keys():
            model[token] = {}
        if END_TOKEN not in model[token].keys():
            model[token][END_TOKEN] = 1
        else:
            model[token][END_TOKEN] += 1 
    
    model[UNKNOWN_TOKEN] = { UNKNOWN_TOKEN: 1 }
    
    if smoothed:
        for key in model.keys():
            for token in vocabulary:
                if token not in model[key].keys():
                    model[key][token] = 0
    
    for key in model.keys():
        total = sum(model[key].values())
        for token in model[key].keys():
            if smoothed:
                model[key][token] = (model[key][token] + 1) / (total + len(vocabulary))
            else:
                model[key][token] = model[key][token] / total
    return model


## Part 3
Implement the function *eval* that returns the perplexity of a model (dictionary) running over the data file of the given target language.

In [5]:
def get_proba_from_to(model: dict, curr_char: str, next_char: str) -> float:
    '''
    Return the probability of the c_to character given the c_from characters
    :param model: the language model
    :param c_from: the n-1 characters
    :param c_to: the n-th character
    :return: the probability of c_to given c_from
    '''
    if curr_char not in model.keys():
        return model[UNKNOWN_TOKEN][UNKNOWN_TOKEN]
    if next_char not in model[curr_char].keys():
        return model[UNKNOWN_TOKEN][UNKNOWN_TOKEN]
    return model[curr_char][next_char]

def eval(model: dict, target_lang: str, n: int) -> float:
    '''
    Return the perplexity value calculated over applying the model on the text file
    of the target_lang language.
    :param model: the language model
    :param target_lang: the target language
    :return: the perplexity value
    '''
    

    tweets = get_texts_for_lang(target_lang)
    proba_tweets = []

    for t in tweets:
        # start of sentence
        for i in range(0,n-1,1):
            curr_chars = START_TOKEN * (n-i-1) + t[0:i]
            next_char = t[i]
            proba_tweets.append(model[curr_chars][next_char])
            
        # n-grams of sentence
        for i in range(len(t)-n+1):
            c_from = t[i:i+n-1]
            c_to = t[i+n-1]
            proba_tweets.append(get_proba_from_to(model,c_from,c_to))
            
        # end of sentence
        c_from = t[i+1:i+n]
        c_to = END_TOKEN
        proba_tweets.append(get_proba_from_to(model,c_from,c_to))

    # print('counts=',len(proba_tweets))
    # print('proba_tweets=',proba_tweets)
    # print('proba_tweets log2=',np.log2(proba_tweets).sum())    
    return np.power(2,- np.log2(proba_tweets).mean())
    

## Part 4
Implement the *match* function that calls *eval* using a specific value of *n* for every possible language pair among the languages we have data for. You should call *eval* for every language pair four times, with each call assign a different value for *n* (1-4). Each language pair is composed of the source language and the target language. Before you make the call, you need to call the *lm* function to create the language model for the source language. Then you can call *eval* with the language model and the target language. The function should return a pandas DataFrame with the following four columns: *source_lang*, *target_lang*, *n*, *perplexity*. The values for the first two columns are the two-letter language codes. The value for *n* is the *n* you use for generating the specific perplexity values which you should store in the forth column.

In [ ]:
def match() -> pd.DataFrame:
  '''
  Return a DataFrame containing one line per every language pair and n_gram.
  Each line will contain the perplexity calculated when applying the language model
  of the source language on the text of the target language.
  :return: a DataFrame containing the perplexity values
  '''
  return pd.DataFrame()

## Part 5
Implement the *generate* function which takes a language code, *n*, the prompt (the starting text), the number of tokens to generate, and *r*, which is the random seed for any randomized action you plan to take in your implementation. The function should start generating tokens, one by one, using the language model of the given source language and *n*. The prompt should be used as a starting point for aligning on the probabilities to be used for generating the next token.

Note - The generation of the next token should be from the LM's distribution.

In [ ]:
def generate(lang: str, n: int, prompt: str, number_of_tokens: int, r: int) -> str:
  '''
  Generate text in the given language using the given parameters.
  :param lang: the language of the model
  :param n: the n_gram value
  :param prompt: the prompt to start the generation
  :param number_of_tokens: the number of tokens to generate
  :param r: the random seed to use
  '''
  return ""

## Part 6
Play with your generate function, try to generate different texts in different language and various values of *n*. No need to submit anything of that.

# Testing

Copy the content of the **tests.py** file from the repo and paste below. This will create the results.json file and download it to your machine.

In [6]:
# Create tests
def test_preprocess():
    return {
        'vocab_length': len(preprocess()),
    }

def test_lm():
    return {
        'english_2_gram_length': len(lm('en', 2, True)),
        'english_3_gram_length': len(lm('en', 3, True)),
        'french_3_gram_length': len(lm('fr', 3, True)),
        'spanish_3_gram_length': len(lm('es', 3, True)),
    }

def test_eval():
    return {
        'en_en': eval(lm('en', 3, True), 'en', 3),
        'en_fr': eval(lm('en', 3, True), 'fr', 3),
        'en_tl': eval(lm('en', 3, True), 'tl', 3),
        'en_nl': eval(lm('en', 3, True), 'nl', 3),
    }

def test_match():
    df = match()
    return {
        'en_en_3': df[(df['source'] == 'en') & (df['target'] == 'en') & (df['n'] == 3)]['perplexity'].values[0],
        'en_tl_3': df[(df['source'] == 'en') & (df['target'] == 'tl') & (df['n'] == 3)]['perplexity'].values[0],
        'en_nl_3': df[(df['source'] == 'en') & (df['target'] == 'nl') & (df['n'] == 3)]['perplexity'].values[0],
    }

def test_generate():
    return {
        'english_1_gram': generate('en', 1, "I", 20, 5), 
        'english_2_gram': generate('en', 2, "I am", 20, 5),
        'english_3_gram': generate('en', 3, "I am", 20, 5),
        'english_4_gram': generate('en', 4, "I Love", 20, 5),
        'spanish_2_gram': generate('es', 2, "Soy", 20, 5),
        'spanish_3_gram': generate('es', 3, "Soy", 20, 5),
        'french_2_gram': generate('fr', 2, "Je suis", 20, 5),
        'french_3_gram': generate('fr', 3, "Je suis", 20, 5),
    }

# TESTS = [test_preprocess, test_lm, test_eval, test_match, test_generate]
TESTS = [test_preprocess, test_lm, test_eval]

# Run tests and save results
res = {}
for test in TESTS:
    try:
        cur_res = test()
        res.update({test.__name__: cur_res})
    except Exception as e:
        res.update({test.__name__: repr(e)})

with open('results.json', 'w') as f:
    json.dump(res, f, indent=2)



In [ ]:
# Show the local files, results.json should be there now and
# also downloaded to your local machine
!ls -l

total 4
drwxr-xr-x 1 root root 4096 Apr  1 13:24 sample_data
